<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/en_whisper_base_test_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x0hj2q1x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-x0hj2q1x
  Resolved https://github.com/huggingface/transformers to commit 0ea42ef0f9f71deba7775ead33afa0e493823d60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=a19a990e4f57dfb8fc078d85a8cfe71dbc761be0e60a9eb0613fc112fb4650a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-yx_32wf9/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "en", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1842
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=6813962ab2b26527a065c021bc24e7354b4894d5774d2e4bc4ca8b576ae3e679
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 173MiB/s]


In [8]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [9]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [10]:
text

,0
0,imposition of switching off the life supportin...
1,for beijing china's best known political priso...
2,the common travel area however does not allow ...
3,the commission has to take strong action befor...
4,there will be clearer rights for travellers wh...
...,...
1837,the only way to halt the breeding ground for s...
1838,other main conclusions of the mission remain o...
1839,i would like to answer you in english.
1840,i wish to reiterate that frontex is one of our...


In [11]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [12]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,imposition of switching off the life supportin...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001708984375, 0.004913330078125, 0.00326538...",16000
1,male,for beijing china's best known political priso...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002655029296875, -0.00030517578125, 0.0018...",16000
2,male,the common travel area however does not allow ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-6.103515625e-05, 0.001312255859375, -0.00192...",16000
3,male,the commission has to take strong action befor...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000274658203125, 0.000823974609375, 0.00027...",16000
4,male,there will be clearer rights for travellers wh...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00115966796875, -0.001739501953125, -0.0030...",16000
...,...,...,...,...,...
1837,male,the only way to halt the breeding ground for s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, 0.00048828125, 0.000122070...",16000
1838,male,other main conclusions of the mission remain o...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005126953125, -0.00250244140625, 0.0003967...",16000
1839,male,i would like to answer you in english.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.20404052734375, -0.2540283203125, -0.28231...",16000
1840,female,i wish to reiterate that frontex is one of our...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001220703125, 0.0003662109375, -0.000213623...",16000


In [13]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,imposition of switching off the life supportin...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001708984375, 0.004913330078125, 0.00326538...",16000
1,male,for beijing china's best known political priso...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002655029296875, -0.00030517578125, 0.0018...",16000
2,male,the common travel area however does not allow ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-6.103515625e-05, 0.001312255859375, -0.00192...",16000
3,male,the commission has to take strong action befor...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000274658203125, 0.000823974609375, 0.00027...",16000
4,male,there will be clearer rights for travellers wh...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00115966796875, -0.001739501953125, -0.0030...",16000
...,...,...,...,...,...
1836,male,let us remind ourselves that historically the ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001953125, -0.000152587890625, 0.0, 0.00170...",16000
1837,male,the only way to halt the breeding ground for s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, 0.00048828125, 0.000122070...",16000
1838,male,other main conclusions of the mission remain o...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.005126953125, -0.00250244140625, 0.0003967...",16000
1839,male,i would like to answer you in english.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.20404052734375, -0.2540283203125, -0.28231...",16000


In [14]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
5,female,two thousand and ten finally we would also urg...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00244140625, -0.004547119140625, 0.0068359...",16000
13,female,we must also strongly condemn the launch of ba...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000701904296875, -0.000457763671875, 3.051...",16000
15,female,mr president the march european council confir...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.016326904296875, 0.01580810546875, 0.018402...",16000
18,female,we have to go and ask for solidarity for the s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, 0.00067138671875, -0.000396728...",16000
22,female,but to do so they expect medical evacuation. w...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.007476806640625, -0.001007080078125, -0.00...",16000
...,...,...,...,...,...
1823,female,who will be responsible?,/root/.cache/huggingface/datasets/downloads/ex...,"[9.1552734375e-05, 0.00018310546875, 9.1552734...",16000
1830,female,uprooting what causes this tragedy is what we ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000946044921875, 0.0, 0.001556396484375, 0....",16000
1832,female,so what can be done?,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0035400390625, 0.0001220703125, 0.00045776...",16000
1833,female,what do the consumers get? legal certainty.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002288818359375, 0.00244140625, 0.000183105...",16000


In [15]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="en")
    return result["text"]

In [16]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [17]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 0.3076923076923077


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.19047619047619047


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 1329 files, WER: 0.0


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1330 files, WER: 0.25


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1331 files, WER: 0.2857142857142857
Average WER: 0.25259301991833655


In [18]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 0.2857142857142857


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.2692307692307692


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 509 files, WER: 0.2


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 510 files, WER: 0.5714285714285714


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 511 files, WER: 0.125
Average WER: 0.2559477562239062
